# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [1]:
import csv
import xlwings as xw
import pandas as pd
import numpy as np
from xlwings import view
recipes1 = pd.read_csv('recipes_sample.csv')
reviews1 = pd.read_csv('reviews_sample.csv')
recipes = pd.DataFrame(data = recipes1, columns=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])
reviews = pd.DataFrame(data = reviews1)
reviews = reviews.rename(columns={"Unnamed: 0": "Indexes"})
print(recipes)
print(reviews)

           id                                          name  minutes  \
0       44123         george s at the cove  black bean soup       90   
1       67664            healthy for them  yogurt popsicles       10   
2       38798                  i can t believe it s spinach       30   
3       35173                          italian  gut busters       45   
4       84797      love is in the air  beef fondue   sauces       25   
...       ...                                           ...      ...   
29995  267661  zurie s holey rustic olive and cheddar bread       80   
29996  386977          zwetschgenkuchen  bavarian plum cake      240   
29997  103312   zwiebelkuchen   southwest german onion cake       75   
29998  486161                                   zydeco soup       60   
29999  298512        cookies by design   cookies on a stick       29   

        submitted                                        description  \
0      2002-10-25  an original recipe created by chef scott mes

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [3]:
wb = xw.Book()
recipes_1 = recipes.take(np.random.permutation(len(recipes))[:(int(5/100 * len(recipes['id'])))])
reviews_1 = reviews.take(np.random.permutation(len(reviews))[:(int(5/100 * len(reviews['Indexes'])))])
print(recipes_1.shape)
with pd.ExcelWriter("recipes.xlsx",engine='xlsxwriter') as writer:
    recipes_1.to_excel(writer, sheet_name="Рецепты")
    reviews_1.to_excel(writer, sheet_name = "Отзывы")

(1500, 6)


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [16]:
file = xw.Book('recipes.xlsx')
wb = file.sheets('Рецепты')
wb.range('H2:H1501').options(transpose=True).value = [i*60 for  i in wb.range('D2:D1501').value]
wb.range('H1').value = ['seconds_assign']

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [12]:
file = xw.Book('recipes.xlsx')
file_1 = file.sheets('Рецепты')
file_1.range('H2:H1501').formula = '=D2*60'
file_1.range('H1').value = ['seconds']

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [19]:
file.sheets[0].range('H1').font.bold = True
wb.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

com_error: (-2147417848, 'Вызванный объект был отключен от клиентов.', None, None)

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [108]:
for i in recipes_1['minutes']:
    for j in range(xw.Range('D2:D1501'):
        if i < 5:
            xw.Range('D2:D1501').color = (0, 128, 0)
        elif i <= 10 and i>=5 :
            xw.Range('D2:D1501').color = (255, 255, 0) 
        else:
            xw.Range('D2:D1501').color = (255, 0, 0)

com_error: (-2146827864, 'OLE error 0x800a01a8', None, None)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [18]:
wb = file.sheets('Рецепты')
wb.range('I2').formula = '=COUNTIF(Отзывы!$C$2:$C$6335;B2)'
wb.range('I1').value = 'n_reviews'

com_error: (-2147417848, 'Вызванный объект был отключен от клиентов.', None, None)

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 